# Параллельные вычисления (2)

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Параллельные вычисления"
* https://nalepae.github.io/pandarallel/
    * https://github.com/nalepae/pandarallel/blob/master/docs/examples_windows.ipynb
    * https://github.com/nalepae/pandarallel/blob/master/docs/examples_mac_linux.ipynb
* https://requests.readthedocs.io/en/latest/
* https://docs.python.org/3/library/pathlib.html
* https://realpython.com/python-pathlib/
* https://realpython.com/python-gil/
* https://docs.python.org/3/library/multiprocessing.html#multiprocessing.pool.ThreadPool

## Лабораторная работа 6

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy` и `pandas`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов `numpy` или структур `pandas` используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

In [4]:
import pandas as pd
from pandarallel import pandarallel
import requests
import json
from tqdm import tqdm
from pathlib import Path

<p class="task" id="1"></p>

1\. Напишите функцию `f`, которая принимает на вход тэг и проверяет, удовлетворяет ли тэг следующему шаблону: `[любое число]-[любое слово]-or-less`. Возьмите файл `tag_nsteps_10m.csv`, примените функцию `f` при помощи метода _серий_ `map` к столбцу `tags` и посчитайте количество тэгов, подходящих под этот шаблон. Выведите количество подходящих тегов на экран. Измерьте время выполнения кода.

In [5]:
import re

In [6]:
def f(tag: str) -> bool:
    if str(tag)[0] in '0123456789':
        if len(re.split(r'\d+', str(tag)))==2:
            return True
        else:
            return False
    else:
        return False

In [7]:
f(str(123))

True

In [8]:
tag_step = pd.read_csv('tag_nsteps_10m.csv')
tag_step

,tags,n_steps
0,camping,4
1,seafood,1
2,whole-chicken,6
3,pasta-rice-and-grains,31
4,pasta,11
...,...,...
9999995,gifts,5
9999996,black-beans,1
9999997,chili,3
9999998,low-in-something,8


In [9]:
%%time
tag_step['tags'].map(f).sum()

Wall time: 8.1 s


302661

<!-- TODO -->

<p class="task" id="2"></p>

2\. Напишите функцию `parallel_map`, которая принимает на вход серию `s` `pd.Series` и функцию одного аргумента `f` и поэлементно применяет эту функцию к серии, распараллелив вычисления при помощи пакета `multiprocessing`. Логика работы функции `parallel_map` должна включать следующие действия:
* разбиение исходной серии на $K$ частей, где $K$ - количество ядер вашего процессора;
* параллельное применение функции `f` к каждой части при помощи метода _серии_ `map` c использованием нескольких подпроцессов;
* объединение результатов работы подпроцессов в одну серию. 

Возьмите файл `tag_nsteps_10m.csv`, примените функцию `f` при помощи `parallel_map` к столбцу `tags` и посчитайте количество тэгов, подходящих под этот шаблон. Выведите количество подходящих тегов на экран. Измерьте время выполнения кода.

In [7]:
from multiprocessing.pool import ThreadPool
import numpy as np

In [8]:
def parallel_map_without(s: pd.Series, f: callable) -> pd.Series:
    data = np.array_split(s, 12)
    return sum(map(lambda x: sum(x.map(f)), data))

In [9]:
%%time
parallel_map_without(tag_step['tags'], f)

Wall time: 9.96 s


302661

In [10]:
def parallel_map(s: pd.Series, f: callable) -> pd.Series:
    data = np.array_split(s, 12)
    with ThreadPool(processes=12) as pool:
        return sum(pool.map(lambda x: sum(x.map(f)), data))

In [11]:
%%time
parallel_map(tag_step['tags'], f)

Wall time: 10.1 s


302661

<p class="task" id="3"></p>

3\. Используя пакет `pandarallel`, примените функцию `f` из задания 1 к столбцу `tags` таблицы, с которой вы работали в этом задании. Посчитайте количество тэгов, подходящих под описанный шаблон. Выведите на экран полученный результат. Измерьте время выполнения кода. 

In [12]:
pandarallel.initialize()

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [13]:
%%time
tag_step['tags'].map(f).sum()

Wall time: 7.65 s


302661

<p class="task" id="4"></p>

4\. Сайт [DummyJSON](https://dummyjson.com/) позволяет получить набор данных о товарах в виде JSON. Воспользовавшись пакетом `requests`, получите данные о __50 товарах__ и создайте словарь, где ключом является название товара (title), а значением - список ссылок на изображения этого товара. При создании словаря замените символ `/` в названии на пробел.

Выведите на экран количество элементов полученного словаря.

In [10]:
r = requests.get("https://dummyjson.com/products?limit=50")
#r.text

In [11]:
d = dict()
for i in r.json()['products']:
    title = i['title'].replace('/',' ')
    d[title] = i['images']
d

{'iPhone 9': ['https://i.dummyjson.com/data/products/1/1.jpg',
  'https://i.dummyjson.com/data/products/1/2.jpg',
  'https://i.dummyjson.com/data/products/1/3.jpg',
  'https://i.dummyjson.com/data/products/1/4.jpg',
  'https://i.dummyjson.com/data/products/1/thumbnail.jpg'],
 'iPhone X': ['https://i.dummyjson.com/data/products/2/1.jpg',
  'https://i.dummyjson.com/data/products/2/2.jpg',
  'https://i.dummyjson.com/data/products/2/3.jpg',
  'https://i.dummyjson.com/data/products/2/thumbnail.jpg'],
 'Samsung Universe 9': ['https://i.dummyjson.com/data/products/3/1.jpg'],
 'OPPOF19': ['https://i.dummyjson.com/data/products/4/1.jpg',
  'https://i.dummyjson.com/data/products/4/2.jpg',
  'https://i.dummyjson.com/data/products/4/3.jpg',
  'https://i.dummyjson.com/data/products/4/4.jpg',
  'https://i.dummyjson.com/data/products/4/thumbnail.jpg'],
 'Huawei P30': ['https://i.dummyjson.com/data/products/5/1.jpg',
  'https://i.dummyjson.com/data/products/5/2.jpg',
  'https://i.dummyjson.com/data/pr

<p class="task" id="5"></p>

5\. Напишите функцию `download_product_imgs`, которая создает папку c названием товара внутри каталога `imgs` (сам каталог `imgs` может быть создан любым удобным способом до начала работы) и сохраняет в нее изображения. Название для файла изображения извлеките из URL.

Воспользовавшись этой функцией, скачайте изображения всех продуктов. Выведите на экран общее количество загруженных файлов. Для отслеживания хода выполнения кода используйте пакет `tqdm`.

In [16]:
# пример кода для скачивания картинки
#url = "https://png.pngtree.com/png-vector/20201229/ourmid/pngtree-a-british-short-blue-and-white-cat-png-image_2654518.jpg"
#img = requests.get(url).content
#with open("imgs/cat.jpg", "wb") as fp:
#    fp.write(img)

In [17]:
def download_product_imgs(title, imgs):
    '''
    title - название товара
    imgs - список ссылок на изображения товара
    '''
    root = Path(f"imgs/{title}")
    root.mkdir(exist_ok=True)
    
    for i in imgs:
        img = requests.get(i).content
        with open(f"imgs/{title}/{i.split('/')[-1]}", "wb") as fp:
            fp.write(img)
            

In [18]:
c = 0

for i in tqdm(d.keys()):
    c += len(d[i])
    download_product_imgs(i,d[i])

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [02:55<00:00,  3.51s/it]


In [19]:
c

223

<p class="task" id="6"></p>

6\. Создайте функцию `download_product_imgs_processes` на основе функции `download_product_imgs`, добавив в нее вывод сообщения следующего вида: `Process ID: <ID текущего процесса>`. Для определения ID процесса воспользуйтесь функцией `multiprocessing.current_process()`.

Решите задачу 5, распараллелив вычисления при помощи процессов. Вместо корневого каталога `imgs` используйте `imgs_processes`. Выведите на экран общее количество загруженных файлов. Измерьте время выполнения кода. 

In [12]:
from pathlib import Path

In [31]:
%%file download_product_imgs_processes.py
def download_product_imgs_processes(data):
    title, imgs = data[0][0], data[0][1]
    import multiprocessing
    from pathlib import Path
    import requests
    root = Path(f"imgs_processes/{title}")
    root.mkdir(exist_ok=True)
    
    for i in imgs:
        img = requests.get(i).content
        with open(f"imgs_processes/{title}/{i.split('/')[-1]}", "wb") as fp:
            fp.write(img)
    print(f'Process ID: {multiprocessing.current_process()}')
    return len(imgs), f'Process ID: {multiprocessing.current_process()}'

Overwriting download_product_imgs_processes.py


In [32]:
import multiprocessing
from download_product_imgs_processes import download_product_imgs_processes 

In [39]:
%%time
with multiprocessing.Pool(processes=8) as pool:
    r = pool.map(download_product_imgs_processes, list(zip(d.items())))
c = 0
for i in r:
    c += i[0]
    print(i[1])
print(c)

Process ID: <SpawnProcess name='SpawnPoolWorker-383' parent=27112 started daemon>
Process ID: <SpawnProcess name='SpawnPoolWorker-383' parent=27112 started daemon>
Process ID: <SpawnProcess name='SpawnPoolWorker-384' parent=27112 started daemon>
Process ID: <SpawnProcess name='SpawnPoolWorker-384' parent=27112 started daemon>
Process ID: <SpawnProcess name='SpawnPoolWorker-385' parent=27112 started daemon>
Process ID: <SpawnProcess name='SpawnPoolWorker-385' parent=27112 started daemon>
Process ID: <SpawnProcess name='SpawnPoolWorker-386' parent=27112 started daemon>
Process ID: <SpawnProcess name='SpawnPoolWorker-386' parent=27112 started daemon>
Process ID: <SpawnProcess name='SpawnPoolWorker-387' parent=27112 started daemon>
Process ID: <SpawnProcess name='SpawnPoolWorker-387' parent=27112 started daemon>
Process ID: <SpawnProcess name='SpawnPoolWorker-388' parent=27112 started daemon>
Process ID: <SpawnProcess name='SpawnPoolWorker-388' parent=27112 started daemon>
Process ID: <Spa